# Project Summary
#### There are many Machine Learning (ML) and Natural Language Processing (NLP) tools to explore. Here I will define several products and analysis goals to achieve with NLP.
* 

In [1]:
%load_ext lab_black

In [2]:
%matplotlib inline

In [25]:
import numpy as np

# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import plotly
import plotly.express as px
import plotly.graph_objects as go
import json
from flask import Flask, jsonify, render_template
import nltk
from nltk.corpus import stopwords
from datetime import date

## Begin with FOTR

In [4]:
# Get the data
# Decision: read from a PDF or .txt file? The PDF may require extra data wrangling and cleanup, which is good, but the .txt can be had from Kaggle. Perhaps better practice from PDF.
# Could use both at first to compare...

In [16]:
PATH = os.path.join("data", "01 - The Fellowship Of The Ring.txt")

In [17]:
with open(
    PATH, "r", encoding="ISO-8859-1"
) as file:  # changed encoding to deal with "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 2836: invalid continuation byte" error
    lotr_text = file.read()

## Experiment with TensorFlow and Keras
#### Think about the end goal and then plot next steps... this might not be the path to go on since this is specific to the Character RNN exercise in the HOML text.

In [18]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [19]:
tokenizer.fit_on_texts([lotr_text])

In [20]:
max_id = len(tokenizer.word_index)

In [21]:
max_id

63

In [22]:
dataset_size = tokenizer.document_count

In [23]:
dataset_size

1

In [24]:
[encoded] = np.array(tokenizer.texts_to_sequences([lotr_text])) - 1

### From news-sentiment-analysis-app

In [ ]:
nltk.download("stopwords")
stoplist = stopwords.words("english")